# CW-Husky Glitch Testing

Runs some of the same glitch tests found in test_husky.py, but (much) longer. Besides the additional validation, this notebook can be helpful to diagnose any issues that may pop up.

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F3'
FORCE_LATEST_BITFILE = False

In [ ]:
if FORCE_LATEST_BITFILE:
    import chipwhisperer as cw

    try:
        if not scope.connectStatus:
            scope.con()
    except NameError:
        scope = cw.scope(name='Husky', bitstream="/home/jpnewae/git/cw_husky/fpga/vivado/cwhusky.runs/impl_no_ilas/cwhusky_top.bit")
        #scope = cw.scope(bitstream="/home/jpnewae/git/cw_husky/fpga/vivado/cwhusky.runs/impl_1/cwhusky_top.bit")

    try:
        if SS_VER == "SS_VER_2_1":
            target_type = cw.targets.SimpleSerial2
        elif SS_VER == "SS_VER_2_0":
            raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
        else:
            target_type = cw.targets.SimpleSerial
    except:
        SS_VER="SS_VER_1_1"
        target_type = cw.targets.SimpleSerial

    try:
        target = cw.target(scope, target_type)
    except:
        print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
        print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
        scope = cw.scope(bitstream="/home/jpnewae/git/cw_husky/fpga/vivado/cwhusky.runs/impl_no_ilas/cwhusky_top.bit")
        target = cw.target(scope, target_type)

    print("INFO: Found ChipWhisperer😍")

    import time
    time.sleep(0.05)
    scope.default_setup()
    def reset_target(scope):
        if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
            scope.io.pdic = 'low'
            time.sleep(0.1)
            scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
            time.sleep(0.1) #xmega needs more startup time
        elif "neorv32" in PLATFORM.lower():
            raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
        else:  
            scope.io.nrst = 'low'
            time.sleep(0.05)
            scope.io.nrst = 'high_z'
            time.sleep(0.05)
    reset_target(scope)

else:
    %run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
scope.fpga_buildtime

In [ ]:
from tqdm.notebook import tnrange
import random
import numpy as np

scope.adc.clip_errors_disabled = True
scope.adc.lo_gain_errors_disabled = True

def reset_setup():
    scope.trigger.module = 'basic'
    scope.trigger.triggers = 'tio4'
    scope.io.tio1 = "serial_rx"
    scope.io.tio2 = "serial_tx"
    scope.io.hs2 = "clkgen"
    scope.adc.timeout = 3
    scope.adc.offset = 0
    scope.glitch.enabled = False
    scope.LA.enabled = False
    scope.LA.clkgen_enabled = True
    scope.LA.capture_depth = 512
    scope.LA.downsample = 1
    scope.trace.enabled = False
    target.baud = 38400 * 10 / 7.37

def setup_glitch(offset, width, oversamp, LA=False):
    # set up glitch:
    scope.glitch.enabled = True
    scope.glitch.clk_src = 'pll'
    scope.clock.pll.update_fpga_vco(600e6)
    scope.glitch.repeat = 1
    scope.glitch.output = 'glitch_only'
    scope.glitch.trigger_src = 'ext_single'
    scope.glitch.offset = offset
    scope.glitch.width = width
    assert scope.glitch.mmcm_locked
    if LA:
        scope.LA.enabled = True
        scope.LA.oversampling_factor = oversamp
        scope.LA.capture_group = 'glitch'
        scope.LA.trigger_source = "glitch_source"
        #scope.LA.trigger_source = "glitch_trigger"
        scope.LA.capture_depth = 512
        assert scope.LA.locked
    else:
        scope.LA.enabled = True

def reset_target():
    scope.io.nrst = 0
    time.sleep(0.2)
    scope.io.nrst = 'high_z'
    time.sleep(0.2)

def find0to1trans(data):
    pattern = [0,1]
    return [i for i in range(0,len(data)) if list(data[i:i+len(pattern)])==pattern]

def test_missing_glitch_sweep_offset(clock, vco, span, width, num_glitches, reps, stepsize, LA=False):
    # Checks for missing glitches (https://github.com/newaetech/chipwhisperer-husky-fpga/issues/4)
    setup_clock(clock)
    scope.clock.pll.update_fpga_vco(vco)
    scope.adc.samples = 16
    errors = []
    for offset in tnrange(scope.glitch.phase_shift_steps//2-span, scope.glitch.phase_shift_steps//2+span, stepsize):
        e = test_missing_glitch_single_offset(offset, width, num_glitches, reps, LA)
        if e:
            errors.append(e)
    assert errors == []

def setup_clock(clock):
    reset_setup()
    scope.clock.clkgen_freq = clock
    scope.clock.adc_mul = 1
    time.sleep(0.1)
    assert scope.clock.pll.pll_locked == True
    assert scope.clock.adc_freq == clock
    target.baud = 38400 * clock / 1e6 / 7.37
    reset_target()

def test_missing_glitch_single_offset(offset, width, num_glitches, reps, LA):
    setup_glitch(offset, width, 1, LA)
    scope.glitch.num_glitches = num_glitches
    scope.io.tio4 = 'high_z'
    errors = []
    for i in range(reps):
        ext_offsets = []
        for j in range(num_glitches):
            ext_offsets.append(random.randrange(2,5))
        scope.glitch.ext_offset = ext_offsets
        scope.glitch.repeat = [1]*num_glitches
        if LA:
            scope.LA.arm()
        trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))
        assert trace is not None, 'capture failed (offset=%d, rep=%d)' % (offset, i)
        if scope.glitch.state != 'idle':
            errors.append(offset)
            print("ERROR: not in idle! state = %s, offset = %d, rep = %d" % (scope.glitch.state, offset, i))
            scope.glitch.state = None
        if LA:
            assert not scope.LA.fifo_empty()
            raw = scope.LA.read_capture_data()
    return errors


In [ ]:
# Test for missing glitches: run sweep around area of interest with lots of repetitions:
# This should take about 15 minutes:
test_missing_glitch_sweep_offset(clock=10e6, vco=600e6, span=100, width=1000, num_glitches=10, reps=5, stepsize=1, LA=True)
#test_missing_glitch_sweep_offset(clock=10e6, vco=1200e6, span=100, width=1000, num_glitches=10, reps=10, stepsize=1)

In [ ]:
scope.glitch.state

In [ ]:
# Test for missing glitches: run sweep around area of interest with lots of repetitions:
# This should take about 40 minutes:
#test_missing_glitch_sweep_offset(clock=10e6, vco=600e6, span=3360, width=1000, num_glitches=10, reps=10, stepsize=1)
test_missing_glitch_sweep_offset(clock=10e6, vco=600e6, span=3360, width=1000, num_glitches=10, reps=1, stepsize=1)

In [ ]:
# example to test a specific offset:
setup_clock(100e6)

In [ ]:
test_missing_glitch_single_offset(offset=1632, width=1000, num_glitches=10, reps=10, LA=True)

In [ ]:
test_missing_glitch_single_offset(offset=1532, width=1000, num_glitches=10, reps=10, LA=True)

In [ ]:
# To investigate behaviour using external logic analyzer:
scope.userio.mode = 'fpga_debug'
scope.fpga_reg_write(109, [4])

Variant with no target attached (useful for testing at clocks > max STM32 clock):

In [ ]:
def test_missing_glitch_sweep_offset_notarget(clock, vco, span, width, oversamp, num_glitches, reps, stepsize):
    # Checks for missing glitches (https://github.com/newaetech/chipwhisperer-husky-fpga/issues/4)
    setup_clock(clock)
    scope.clock.pll.update_fpga_vco(vco)
    scope.adc.samples = 16
    errors = []
    for offset in tnrange(scope.glitch.phase_shift_steps//2-span, scope.glitch.phase_shift_steps//2+span, stepsize):
        e = test_missing_glitch_single_offset_notarget(offset, width, oversamp, num_glitches, reps)
        if e:
            errors.append(e)
    assert errors == []

def test_missing_glitch_single_offset_notarget(offset, width, oversamp, num_glitches, reps):
    #offset = 1632 # 10M
    #num_glitches = 1
    #reps = 100
    #oversamp = 4
    #LA = True

    scope.io.tio4 = 0
    assert scope.io.tio_states[3] == 0, 'This will not work if IO4 is not cleared.'
    setup_glitch(offset, width, oversamp, True)
    scope.glitch.trigger_src = 'manual'
    scope.glitch.num_glitches = num_glitches
    errors = []
    for i in range(reps):
        ext_offsets = []
        for j in range(num_glitches):
            ext_offsets.append(random.randrange(2,5))
        scope.glitch.ext_offset = ext_offsets
        scope.glitch.repeat = [1]*num_glitches
        scope.LA.arm()
        scope.glitch.manual_trigger()
        assert not scope.LA.fifo_empty()
        raw = scope.LA.read_capture_data()
        glitchenable = scope.LA.extract(raw, 6)
        glitchenlen = len(np.where(glitchenable == 1)[0])
        if glitchenlen == 0:
            errors.append(offset)
            print('Offset %d, iteration %d: Expected glitch length = %d, got %d' % (offset, i, oversamp, glitchenlen))
    scope.io.tio4 = 'high_z' # return to default
    return errors


In [ ]:
test_missing_glitch_single_offset_notarget(1633, 1000, 10, 20)

In [ ]:
test_missing_glitch_sweep_offset_notarget(clock=50e6, vco=600e6, span=scope.glitch.phase_shift_steps, width=500, num_glitches=1, reps=20, stepsize=1)

In [ ]:
scope.trace.clock._warning_frequency = 401e6

In [ ]:
test_missing_glitch_sweep_offset_notarget(clock=100e6, vco=600e6, span=scope.glitch.phase_shift_steps, width=500, num_glitches=1, reps=20, stepsize=1)

In [ ]:
scope.glitch.phase_shift_steps

In [ ]:
scope.clock.pll.update_fpga_vco(1200e6)

In [ ]:
test_missing_glitch_sweep_offset_notarget(clock=100e6, vco=1200e6, span=scope.glitch.phase_shift_steps, width=500, oversamp=4, num_glitches=1, reps=20, stepsize=1)

In [ ]:
scope.clock.adc_freq

In [ ]:
scope.fpga_buildtime

In [ ]:
errors

In [ ]:
glitchout    = scope.LA.extract(raw, 0)
source       = scope.LA.extract(raw, 1)
mmcm1out     = scope.LA.extract(raw, 2)
mmcm2out     = scope.LA.extract(raw, 3)
glitchgo     = scope.LA.extract(raw, 4)
glitchenable = scope.LA.extract(raw, 6)
glitchsource = scope.LA.extract(raw, 7)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span, Legend, LegendItem
import numpy as np
output_notebook(INLINE)

o = figure(plot_width=1800)

xrange = range(len(source))
O1 = o.line(xrange, source + 6, line_color='black')
O2 = o.line(xrange, mmcm1out + 4, line_color='blue')
O3 = o.line(xrange, mmcm2out + 2, line_color='red')
O4 = o.line(xrange, glitchout + 0, line_color='purple', line_width=2)
O5 = o.line(xrange, glitchenable - 2, line_color='black', line_width=2)
O6 = o.line(xrange, glitchgo - 4, line_color='green')
O7 = o.line(xrange, glitchsource - 6, line_color='pink', line_width=2)

legend = Legend(items=[
    LegendItem(label='source clock', renderers=[O1]),
    LegendItem(label='glitch MMCM1 output (internal signal)', renderers=[O2]),
    LegendItem(label='glitch MMCM2 output (internal signal)', renderers=[O3]),
    LegendItem(label='glitch clock output', renderers=[O4]),
    LegendItem(label='glitch enable', renderers=[O5]),
    LegendItem(label='glitch go', renderers=[O6]),
    LegendItem(label='glitch trigger source', renderers=[O7]),
])
o.add_layout(legend)

In [ ]:
show(o)

In [ ]:
len(np.where(glitchenable == 1)[0])

In [ ]:
scope.glitch.state

In [ ]:
scope.glitch.manual_trigger()

In [ ]:
scope.glitch.state

In [ ]:
scope.glitch.state = None

In [ ]:
scope.glitch